// Reset Query 

// Query to delete all edges and nodes, just using APOC to speed up process 

CALL apoc.periodic.iterate(  
    "MATCH (n) RETURN n",  
    "DETACH DELETE n",  
    {batchSize:250}  
    )

// creating song nodes

// loading in the data
LOAD CSV WITH HEADERS FROM 'file:///experimentalv2_spotify.csv' AS line

// only keeping relevant information to the track wanted in nodes 
 
CREATE (:Song {  
    track_id: line.track_id,  
    track_name: line.track_name,  
    artists: line.artists,  
    album_name: line.album_name,  
    track_genre: line.track_genre  
    })

// creating relationships (edges)

// APOC used to maximize edge count without failure  
CALL apoc.periodic.iterate( "LOAD CSV WITH HEADERS FROM 'file:///experimentalv2_relationships.csv' AS line RETURN line", "WITH line  

// filtering out to only create relationships (edges) where the distance is less than the threshold  
// to try and only capture relevant recommendations and adhearing to computers limit, set reasonable threshold -- found 0.25 for 10k nodes, 0.2 for 25k nodes 

WHERE toFloat(line.Distance) < $threshold  
MATCH (song1:Song {track_id: line.SongID1})  
MATCH (song2:Song {track_id: line.SongID2})  
MERGE (song1)-[r:SIMILAR_TO]->(song2)  
ON CREATE SET r.distance = toFloat(line.Distance)",  
// set threshold here  
{batchSize: 200, params: {threshold: 0.2}}  
)

### We uploaded the pngs to the submission file, so either download and view here, or view in submission - sorry for inconvenience!

<img src="setup.png">

// Algorithm - specific testing for hw5 testing with Is This It album (Query result)

// get the album songs, the distances, and another song to check against  
MATCH (albumSong:Song)-[r:SIMILAR_TO]->(otherSong:Song)

// filtering to get songs from correct album, then make sure it contains The Strokes  
// but also that the recommendation song cannot be from The Strokes  
WHERE albumSong.album_name = "Is This It" AND albumSong.artists CONTAINS "The Strokes" AND NOT otherSong.artists = "The Strokes"  
WITH albumSong, otherSong, r.distance AS similarity

// limiting by 5 to get 5 recommendations and ordering the similarity score ascending to get the closest distances to 0  
// meaning more similar  
ORDER BY similarity ASC  
LIMIT 5

// returning the album song, the song that is similar to that album song, and the distance between the two  
RETURN DISTINCT albumSong, similarity, otherSong

### We uploaded the pngs to the submission file, so either download and view here, or view in submission - sorry for inconvenience!

<img src="recommendation result.png">

// Run parameters first and then run query below to get recommendations for any artist, album, or specific song

// Put null wherever you are not looking, so if looking for recommendations for specific song, put "song name" and null for other etc.  
:param songName => null;  
:param artist => null;  
:param albumName => "Is This It";

// General Recommendation Engine

// Initialize parameters above -- retrieving parameters here  
WITH COALESCE($songName, '') AS songName, COALESCE($artist, '') AS artist, COALESCE($albumName, '') AS albumName

// matching by song  
OPTIONAL MATCH (song:Song)  
WHERE songName <> '' AND song.track_name = songName

// matching by artist (only if no specific song was given)  
OPTIONAL MATCH (artistSong:Song)  
WHERE songName = '' AND artist <> '' AND artistSong.artists CONTAINS artist

// matching by album (only if no specific song or artist was given)  
OPTIONAL MATCH (albumSong:Song)  
WHERE songName = '' AND artist = '' AND albumName <> '' AND albumSong.album_name = albumName

// combine all matches and remove nulls  
WITH COLLECT(song) + COLLECT(artistSong) + COLLECT(albumSong) AS allMatches  
UNWIND allMatches AS matchedSong  
WITH matchedSong WHERE matchedSong IS NOT NULL

// find similar songs based on the matched songs, excluding the same artist  
MATCH (matchedSong)-[r:SIMILAR_TO]->(otherSong:Song)  
WHERE NOT otherSong.artists CONTAINS matchedSong.artists  
WITH matchedSong, otherSong, r  

// ordering by distance ascending to get songs with relationship distance closest to 0 (closer to 0 = more similar)  
// and limiting by 5 as it was asked for 5 recommendations
ORDER BY r.distance ASC  
LIMIT 5  

// returning recommended song, distance and the song which the recommendation was matched with
RETURN DISTINCT matchedSong AS Input, r, otherSong AS RecommendedTrack

### We uploaded the pngs to the submission file, so either download and view here, or view in submission - sorry for inconvenience!

<img src="parameterized recommendation result.png">